In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as Layers
import tensorflow.keras.activations as Activations
import tensorflow.keras.models as Models
import tensorflow.keras.optimizers as Optimizer
import keras 
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import time
import pandas as pd
from sklearn.utils import shuffle
from IPython.display import SVG
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import accuracy_score

In [ ]:
class_names = ['mountain', 'street', 'glacier', 'buildings', 'sea', 'forest'] 
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}  
nb_classes = len(class_names)  
IMAGE_SIZE = (32, 32)
IMAGE_SIZE1 = (150, 150)  
# Image sizes used are different for different algoritms.
# for NN and CNN size is 64*64
# for InceptionV3 and VGG are 150*150
# for other algoritmhms is 32*32

In [ ]:
def load_data():
    datasets = ['../input/intel-image-classification/seg_train/seg_train', '../input/intel-image-classification/seg_test/seg_test']
    output = []
   
    # Iterate through training and test sets
    for dataset in datasets:
       
        images = []
        labels = []
       
        print("Loading {}".format(dataset))
       
        # Iterate through each folder corresponding to a category
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
           
            # Iterate through each image in our folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
               
                # Get the path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
               
                # Open and resize the img
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, IMAGE_SIZE)
               
                # Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
               
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')  
        print(images.shape)
        output.append((images, labels))

    return output


In [ ]:
def load_data1():
   
    datasets = ['../input/intel-image-classification/seg_train/seg_train', '../input/intel-image-classification/seg_test/seg_test']
    output = []
   
    # Iterate through training and test sets
    for dataset in datasets:
       
        images = []
        labels = []
       
        print("Loading {}".format(dataset))
       
        # Iterate through each folder corresponding to a category
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
           
            # Iterate through each image in our folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
               
                # Get the path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
               
                # Open and resize the img
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, IMAGE_SIZE1)
               
                # Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
               
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')  
        print(images.shape)
        output.append((images, labels))

    return output


In [ ]:
(train_images, train_labels), (X_test, y_test) = load_data()  
X_train, y_train = shuffle(train_images, train_labels, random_state=25)

In [ ]:
(train_images1, train_labels1), (Xnn_test, ynn_test) = load_data1()  
Xnn_train, ynn_train = shuffle(train_images1, train_labels1, random_state=25)

In [ ]:
# scaling is done by dividing each value by 255 and each value will be in range 0 and 1
# for CNN NN InceptionV3 and VGG input are 4D arrays
# for others are 2D array

Xnn_train = Xnn_train/255
Xnn_test = Xnn_test/255
print(Xnn_train.shape)

In [ ]:
X_train = X_train.reshape(X_train.shape[0],32*32*3) 
X_test = X_test.reshape(X_test.shape[0],32*32*3) 
X_train = X_train/255; 
X_test = X_test/255;


In [ ]:
# Code for logistic regression
from sklearn.linear_model import LogisticRegression
tic = time.time()
log_reg = LogisticRegression(solver='lbfgs',multi_class='multinomial',max_iter = 10000)
log_reg.fit(X_train,y_train)
predlr = log_reg.predict(X_test)
print(accuracy_score(predlr,y_test))
toc = time.time()
LRtime = toc-tic
print(LRtime)

In [ ]:
# code for naive bayes with linear 
from sklearn.naive_bayes import MultinomialNB
tic = time.time()
NB = MultinomialNB()
NB.fit(X_train,y_train)
prednb = NB.predict(X_test)
print(accuracy_score(prednb,y_test))
toc = time.time()
NBtime = toc-tic
print(NBtime)

In [ ]:
# code for naive bayes with gaussian
from sklearn.naive_bayes import GaussianNB
tic = time.time()
GNB = GaussianNB()
GNB.fit(X_train,y_train)
predgnb = GNB.predict(X_test)
print(accuracy_score(predgnb,y_test))
toc = time.time()
NBtime = toc-tic
print(NBtime)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tic = time.time()
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)
preddt = dt.predict(X_test)
print(accuracy_score(preddt,y_test))
toc = time.time()
KNNtime = toc-tic
print(KNNtime)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
tic = time.time()
knn = KNeighborsClassifier(n_neighbors = 6)
knn.fit(X_train,y_train)
predknn = knn.predict(X_test)
print(accuracy_score(predknn,y_test))
toc = time.time()
KNNtime = toc-tic
print(KNNtime)

In [ ]:
from keras.models import Sequential
tic = time.time()
model2 =Sequential([
    keras.layers.Flatten( input_shape = (64,64,3)),
    keras.layers.Dense(64*3, activation= 'relu' ),
    keras.layers.Dense(64, activation= 'relu' ),
    keras.layers.Dense(32, activation= 'relu' ),
    keras.layers.Dense(6, activation= 'softmax')
    
])

model2.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

history2 = model2.fit(Xnn_train, ynn_train, batch_size=128, epochs=16, validation_split = 0.2)

toc = time.time()
NNtime = toc-tic
print(NNtime)


In [ ]:
model2.evaluate(Xnn_test,ynn_test)


In [ ]:
tic = time.time()
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu', input_shape = (64, 64, 3)), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(6, activation=tf.nn.softmax)
])
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(Xnn_train, ynn_train, batch_size=128, epochs=10, validation_split = 0.2)
toc = time.time()
CNNtime = toc-tic
print(CNNtime)

In [ ]:
model.evaluate(Xnn_test,ynn_test)

In [ ]:
tic = time.time()
import xgboost as xgb
xg_cl = xgb.XGBClassifier(objective='multi:softprob',
n_estimators=10, seed=123)
xg_cl.fit(X_train, y_train)
preds = xg_cl.predict(X_test)
accuracy = float(np.sum(preds==y_test))/y_test.shape[0]
print(accuracy)
toc = time.time()
XGBtime = toc-tic
print(XGBtime)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
tic = time.time()
clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)  
y_pred=clf.predict(X_test)
from sklearn import metrics  
print("Accuracy:",metrics.accuracy _score(y_test, y_pred))
toc = time.time()
RFtime = toc-tic
print(RFtime)

In [ ]:
from sklearn.decomposition import PCA
tic = time.time()
pca = PCA(n_components=500)
pca.fit_transform(X_train)
print(pca.explained_variance_ratio_.sum())

In [ ]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train,y_train)
predsvm = svm.predict(X_test)
toc = time.time()
svmtime = toc-tic
print(svmtime)


In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, predsvm))


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop

local_weights_file = '/kaggle/input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
     layer.trainable = False
        


last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense(6, activation='softmax')(x)           

model = Model(pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['acc'])

history=model.fit(Xnn_train,ynn_train,epochs=1,validation_split = 0.2)

In [ ]:
model.evaluate(Xnn_test,ynn_test)

In [ ]:
from tensorflow.keras.applications import VGG16
file='/kaggle/input/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
pretrained_model=VGG16(input_shape = (150, 150, 3), 
                        include_top = False, 
                        weights =None)
pretrained_model.load_weights(file)

for layer in pretrained_model.layers:
     layer.trainable = False

last_layer = pretrained_model.get_layer('block5_pool')
print('last layer of vgg : output shape: ', last_layer.output_shape)
last_output = last_layer.output

y = layers.Flatten()(last_output)
y = layers.Dense(1024, activation='relu')(y)
y = layers.Dropout(0.2)(y)                  
y = layers.Dense(6, activation='softmax')(y)           

model_vgg = Model(pretrained_model.input, y) 


model_vgg.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['acc'])



In [ ]:
model_vgg.fit(Xnn_train,ynn_train,epochs=1,validation_split=0.2)